In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
import re

import wandb
from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import *

In [80]:
# TODO for now only char-level syllabification
raw_text = open("./data/divina_textonly.txt", "rb").read().decode(encoding="utf-8")
raw_syll_text = (
    open("./data/divina_syll_textonly.txt", "rb").read().decode(encoding="utf-8")
)
syll_text = preprocess_tercets(raw_syll_text)
text = preprocess_tercets(raw_text)

Split preprocessed text into tercets

In [102]:
sep = "<EOT>"
input_tercets = [x + sep for x in text.split(sep)][:-1]
target_tercets = [x + sep for x in syll_text.split(sep)][:-1]

Encode with tokenizer

In [107]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False, filters="", lower=False
)
tokenizer.fit_on_texts(target_tercets)
enc_input_tercets = tokenizer.texts_to_sequences(input_tercets)
enc_target_tercets = tokenizer.texts_to_sequences(target_tercets)
vocab_size = len(tokenizer.word_index) + 1

Pad sequences

In [125]:
input_train = tf.keras.preprocessing.sequence.pad_sequences(
    enc_input_tercets, padding="post"
)
target_train = tf.keras.preprocessing.sequence.pad_sequences(
    enc_target_tercets, padding="post"
)

In [139]:
sweep_config = {
    "name": "sweep-test-1",
    "method": "random",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "batch_size": {"values": [32, 64]},
        "epochs": {"values": [10, 20, 50, 100]},
        "num_layers": {"values": [4, 6, 8, 12]},
        "num_heads": {"values": [2, 4, 8]},
        "d_model": {"values": [256, 512]},
        "dff": {"values": [512, 1024, 2048]},
        # TODO include architecture + dataset
    },
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 7f44jv1h
Sweep URL: https://wandb.ai/alessandropacielli/uncategorized/sweeps/7f44jv1h


In [140]:
def make_dataset(input_tercets, target_tercets, batch_size):
    buffer_size = len(input_tercets)
    batch_size = 64

    steps_per_epoch = len(input_tercets) // batch_size

    dataset = tf.data.Dataset.from_tensor_slices(
        (input_tercets, target_tercets)
    ).shuffle(buffer_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [141]:
def make_model(config):
    transformer = Transformer(
        num_layers=config["num_layers"],
        d_model=config["d_model"],
        num_heads=config["num_heads"],
        dff=config["dff"],
        input_vocab_size=vocab_size,
        target_vocab_size=vocab_size,
        pe_input=1000,
        pe_target=1000,
        rate=0.1,
    )
    transformer_trainer = TransformerTrainer(transformer, checkpoint_save_path=None)

    return transformer, transformer_trainer

In [142]:
def sweep():
    with wandb.init() as run:
        config = wandb.config
        dataset = make_dataset(input_train, target_train, config["batch_size"])
        model, trainer = make_model(config)
        loss = trainer.train(dataset, config["epochs"])
        wandb.log({"loss": loss, "epoch": epoch})


count = 5
wandb.agent(sweep_id, function=sweep, count=count)

wandb: Agent Starting Run: 5wrukb8l with config:
wandb: 	batch_size: 32
wandb: 	d_model: 256
wandb: 	dff: 2048
wandb: 	epochs: 10
wandb: 	num_heads: 2
wandb: 	num_layers: 6
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Ctrl + C detected. Stopping sweep.
